In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Price Volume data retrieval using Yahoo Finance API

Target tickers:
- S&P 500 Index (SPY)
- VIX Index (^VIX)
- Gold (GC=F)
- Brent Crude (BZ=F)
- WTI (CL=F)

In [2]:
# !apt-get install -y build-essential
# !apt-get install -y wget
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib/
# !./configure --prefix=/usr
# !make
# !make install
#!pip uninstall -y numpy
#!pip install numpy == 1.20.3
#!pip install TA-Lib

import yfinance as yf
import talib

In [3]:
# Create price extractor

class price_volume_data:
    """A class for retrieving and analyzing price and volume data."""
    def __init__(self, tickers, start_date, end_date):
        """
        Initializes a new instance of the price_volume_data class.

        Args:
            tickers (list): A list of tickers for the assets to retrieve data for.
            start_date (str): The start date for the data retrieval (format: 'YYYY-MM-DD').
            end_date (str): The end date for the data retrieval (format: 'YYYY-MM-DD').
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.pv = self.__retrieve_data__()
        #self.lp, self.ret = self.__get_daily_returns__() #didn't need daily returns
        
    def __retrieve_data__(self):
        """
        Retrieves price and volume data for the given tickers from Yahoo Finance API.

        Returns:
            dict: A dictionary containing the price and volume data for each ticker.
        """

        data, failed = {}, []
        for ticker in self.tickers:
            try:
                df = yf.Ticker(ticker).history(start = self.start_date, end = self.end_date)
                
                data[ticker] = df
                
            except:
                failed.append(ticker)
                
        if failed:
            print('Tickers failed to extract: ', failed)
        
        return data
    
    def __get_daily_returns__(self):
        """
        Computes daily returns for the retrieved price data.

        Returns:
            pd.DataFrame: A DataFrame containing the daily price data.
            pd.DataFrame: A DataFrame containing the daily price returns.
        """
        df = pd.DataFrame()
        
        for ticker in self.pv:
            if df.empty:
                df = pd.DataFrame(self.pv[ticker]['Close'].copy())
            else:
                df = df.merge(self.pv[ticker]['Close'], how='outer',left_index=True, right_index=True)

        df.columns = self.tickers
        df = df.fillna(method = 'ffill')
        
        return df, df/df.shift(1) - 1
    
    def get_info(self, ticker = None):
        """
        Prints information about the price and volume data for the specified ticker.
        If no ticker is provided, prints information for all tickers.

        Args:
            ticker (str, optional): The ticker symbol for the asset. Defaults to None.
        """
        if ticker:
            self.pv[ticker].info()
        
        else:
            for ticker in self.pv:
                self.pv[ticker].info()
                
        return None
    
    def plot_price_trends(self, normalize = True, reference = -1):
        """
        Plots the close prices of selected assets over time.

        Args:
            normalize (bool, optional): Indicates whether to normalize the prices. Defaults to True.
            reference (int, optional): Reference point for normalization. Defaults to -1.
        """
        f, ax = plt.subplots(1,1,figsize=(12,5))
        # plot prices
        for key in self.pv:
            last_price = self.pv[key].loc[:, 'Close'].copy()
            if normalize:
                if reference == -1:
                    last_price /= last_price.iloc[0]
                elif reference == 0:
                    midpt = int(last_price.shape[0]/2)
                    last_price /= last_price.iloc[midpt]
                else:
                    last_price /= last_price.iloc[-1]
            last_price.plot(ax = ax, label = key)
        
        ax.title.set_text('Close Prices of selected assets')
        ax.set(ylabel = 'Normalized Price to start of observation')
        ax.legend()
        plt.show()
        
    def plot_corr(self, method='pearson', output = False, series = 'price'):
        """
        Plots the correlation matrix between asset prices or returns.

        Args:
            method (str, optional): The correlation method to use. Defaults to 'pearson'.
            output (bool, optional): Indicates whether to return the correlation matrix. Defaults to False.
            series (str, optional): Indicates whether to use 'price' or 'returns'. Defaults to 'price'.

        Returns:
            pd.DataFrame or None: The correlation matrix if output=True, None otherwise.
        """
        corr_matrix = self.lp.corr(method) if series == 'price' else self.ret.corr(method)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        f, ax = plt.subplots(figsize=(5, 5))
        sns.heatmap(corr_matrix, mask=mask, vmin=-1, vmax=1, annot=True)
        
        return corr_matrix if output else None
        
        
    def get_technical_indicators(self, indicators = None, normalize = True):
        """
        Computes and adds various technical indicators to the price data.

        Args:
            indicators (list, optional): A list of tuples specifying the indicators and their parameters.
                Defaults to None.
            normalize (bool, optional): Indicates whether to normalize the indicator values. Defaults to True.
        """
        if not indicators:
            indicators = [('SMA', 5), ('SMA', 14), ('ATR', 5), ('ATR', 14), ('ADX', 5),
                         ('ADX', 14), ('RSI', 5), ('RSI', 14), ('MACD', None), ('BBANDS', None)]
        
        for data in self.pv:
            df = self.pv[data]
            for indicator, param in indicators:
                
                if indicator == 'SMA':
                    df[f'{indicator}_{param}'] = talib.SMA(df['Close'], param)

                elif indicator == 'ATR':
                    df[f'{indicator}_{param}'] = talib.ATR(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'ADX':
                    df[f'{indicator}_{param}'] = talib.ADX(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'RSI':
                    df[f'{indicator}_{param}'] = talib.RSI(df['Close'], param) 
                    
                elif indicator == 'MACD':
                    df[f'{indicator}'], df[f'{indicator}_signal'], _ = talib.MACD(df['Close']) 
                    
                    
                elif indicator == 'BBANDS':
                    df[f'{indicator}_upp'], df[f'{indicator}_mid'], df[f'{indicator}_low'] = talib.BBANDS(df['Close'])
                    
                    
                else:
                    print('Indicators not built into the class. Please run TALIB manually!')
            
            df = df.replace([np.inf, -np.inf], np.nan, inplace = True)
            
        return None              

In [4]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM
from keras.regularizers import L1, L1L2, L2
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

import numpy as np
import tensorflow as tf
import random as python_random

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

def reset_seeds(SEED = 624):
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

tf.config.experimental.enable_op_determinism()

AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

In [ ]:
class LSTM_predictor:
    """A class for predicting time series using LSTM (Long Short-Term Memory) models."""
    def __init__(self, X, Y):
        """
        Initializes a new instance of the LSTM_predictor class.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.
        """
        self.X, self.Y = self.__clean_data__(X, Y)
        self.features_size = X.shape[1]
        self.obs_size = X.shape[0]
        
        
    def __clean_data__(self, X, Y):
        """
        Cleans the input feature matrix and target values by removing rows with NaN values.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.

        Returns:
            ndarray: The cleaned input feature matrix.
            ndarray: The cleaned target values.
        """
        Z = np.concatenate((X, Y), axis = 1)
        Z = Z[~np.isnan(Z).any(axis=1), :]
        return Z[:, :-1], Z[:, -1]
        
    def prepare_data(self, train_test_split = 0.8, timestep = 3, look_forward = 1, inplace = False, scaled = True):
        """
        Prepares the data for training and testing the LSTM model.

        Args:
            train_test_split (float, optional): The fraction of data to be used for training.
                Defaults to 0.8.
            timestep (int, optional): The number of time steps to consider for each sample.
                Defaults to 3.
            inplace (bool, optional): Indicates whether to store the prepared data in the class
                instance or return it. Defaults to False.
            scaled (bool, optional): Indicates whether to scale the data. Defaults to True.

        Returns:
            tuple or None: If inplace=True, returns None. Otherwise, returns a tuple containing
            the prepared training and testing data as (X_train, Y_train, X_test, Y_test).
        """
        split_idx = int(self.obs_size*0.8)
        train_set = self.X[:-4,:], self.Y[:-4]
        test_set = self.X[-4,:], self.Y[-4]
        
        # Scale factors
        mean_X = np.mean(train_set[0], axis = 0)
        std_X = np.std(train_set[0], axis = 0)
        self.__scale_X__ = (mean_X, std_X)
        
        mean_Y = np.mean(train_set[1], axis = 0)
        std_Y = np.std(train_set[1], axis = 0)
        self.__scale_Y__ = (mean_Y, std_Y)
        
        sampled = []
        
        for X, Y in [train_set, test_set]:
            X_sampled, Y_sampled = [], []

            for i in range(len(X)-timestep-look_forward+1):
                X_sampled.append((X[i:i+timestep] - self.__scale_X__[0])/self.__scale_X__[1])
                Y_sampled.append((Y[i+timestep+look_forward-1] - self.__scale_Y__[0])/self.__scale_Y__[1])
            
            sampled.append((np.array(X_sampled),np.array(Y_sampled)))
        
        if inplace:
            self.X_train, self.Y_train = sampled[0]
            self.X_test, self.Y_test = test_set
            print("x_train after prepare_data has shape of {}".format(self.X_train.shape))
            print("x_test after prepare_data has shape of {}".format(self.X_test.shape))
            
            return None
        else:
            return sampled
        
    def init_model(self, optimizer='adam', loss='mean_squared_error', kernel_regularizer=None, num_lstm_units = 64, num_lstm_units_2 = 64,num_dense_units = 25, learning_rate=0.001):
        """
        Initializes the LSTM model with the specified optimizer and loss function.

        Args:
            optimizer (str, optional): The optimizer to use. Defaults to 'adam'.
            loss (str, optional): The loss function to use. Defaults to 'mean_squared_error'.
        """
        reset_seeds()
        model = Sequential()
        model.add(LSTM(num_lstm_units, return_sequences=True,\
                       input_shape= (self.X_train.shape[1], self.features_size),\
                      kernel_regularizer=kernel_regularizer, activation = 'relu')) #added activation = "relu"
        model.add(LSTM(num_lstm_units_2, return_sequences=False, kernel_regularizer=kernel_regularizer, activation = 'relu'))
        model.add(Dense(num_dense_units))
        model.add(Dense(1))
        
        
        #opt = Adam(learning_rate=learning_rate) #to remove and default to the code directly below
        
        #if optimizer == 'adam':
            #opt = Adam(learning_rate=learning_rate)
        #elif optimizer == 'SGD':
            #opt = SGD(learning_rate=learning_rate)
        #else:
            #raise ValueError(f"Unsupported optimizer: {optimizer}")
        
        model.compile(optimizer = optimizer, loss = loss) # changed optimizer = optimizer to optimizer = opt
        self.model = model
        
        return None
    
    def fit_model_train(self, batch_size=100, epochs=50, callbacks=None):
        """
        Trains the LSTM model.

        Args:
            batch_size (int, optional): The batch size for training. 
            epochs (int, optional): The number of epochs to train for. 
        """
        self.test_loss_list = []
        self.pred_per_epoch = []
        history_train = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, verbose=1, callbacks = callbacks)
        self.train_loss_list = history_train.history['loss']  # Store loss values in the list 
       
        
    def predict(self):
        """Makes predictions using the trained LSTM model."""
        self.Y_pred = self.model.predict(self.X[-4,:])
        #mse = mean_squared_error(self.Y_test, self.Y_pred)
        #mse_list.append(mse)
        
    def plot_prediction(self, epoch = None):
        """Plots the predicted and actual values."""
        if epoch == None:
            Y_pred = self.Y_pred
        else:
            Y_pred = self.pred_per_epoch[epoch - 1]
        actual = np.concatenate((self.Y_train.reshape(-1,1), self.Y_test.reshape(-1,1)))
        actual = actual*self.__scale_Y__[1] + self.__scale_Y__[0]
        prediction = np.concatenate((self.Y_train.reshape(-1,1), Y_pred))
        prediction = prediction*self.__scale_Y__[1] + self.__scale_Y__[0]
        
        f, ax = plt.subplots(figsize=(12,5))
        plt.plot(prediction, label = 'Predicted')
        plt.plot(actual, label = 'Actual')
        
        ax.title.set_text('Close Prices of selected assets')
        ax.set(ylabel = 'Normalized Price to start of observation')
        ax.legend()
        plt.show()
        
    def key_metrics(self,epoch=None):
        Y_pred = self.Y_pred if epoch == None else self.pred_per_epoch[epoch-1]
        mse = mean_squared_error(self.X[-1,:], Y_pred)
        mae = mean_absolute_error(self.Y[-1], Y_pred)
        exp_ratio = explained_variance_score(self.Y_test, Y_pred)
        
        return (Y_pred[-1]*self.__scale_Y__[1] + self.__scale_Y__[0]), mse, mae, exp_ratio 
    
    def cross_validation(self, n_splits = 5, batch_size=100, epochs=10, callback=True):
        """
        Perform cross-validation for the LSTM model.

        Parameters:
        - n_splits: Number of cross-validation splits (int, default=5)
        - batch_size: Number of samples per gradient update (int, default=100)
        - epochs: Number of epochs to train the model (int, default=50)
        - callback: Flag indicating whether to use custom callback during training (bool, default=True)

        Returns:
        - None
        """
        reset_seeds()
        tscv = TimeSeriesSplit(n_splits=5)
        
        cv_train_scores = []
        cv_val_scores = []
        
        print('Initiate cross-validation')
        i = 1

        for train_index, test_index in tscv.split(self.X_train):
            print(f'Running cross validation round {i}')
            self.temp_val_scores = []

            #print(self.X_train[train_index].shape)
            #print(self.X_train[test_index].shape)
            
            cv_X_train, cv_X_test = self.X_train[train_index], self.X_train[test_index]
            cv_y_train, cv_y_test = self.Y_train[train_index], self.Y_train[test_index]
            
            #print(cv_X_test.shape)
    
            # Fit your model on the training data
            history_train = self.model.fit(cv_X_train, cv_y_train, batch_size=batch_size,\
                                           epochs=epochs,\
                                           callbacks = [CustomCallback(self,cv_X_test, cv_y_test, cv = True)],
                                          verbose = 2)
            cv_train_scores.append(history_train.history['loss'])
            cv_val_scores.append(self.temp_val_scores)
            print('==============================================')
            i+=1
            
        self.cv_train_scores = np.array(cv_train_scores).mean(axis = 0)
        self.cv_val_scores = np.array(cv_val_scores).mean(axis = 0)

class CustomCallback(keras.callbacks.Callback):
    """
    CustomCallback class for defining custom callbacks during model training.
    """
    def __init__(self, model_class, X_test = np.empty(0), Y_test = np.empty(0), cv = False):
        """
        Initialize the CustomCallback object.

        Parameters:
        - model_class: Instance of the LSTM_predictor class
        - X_test: Test input features (numpy array, default=np.empty(0))
        - Y_test: Test target variable (numpy array, default=np.empty(0))
        - cv: Flag indicating whether it is a cross-validation callback (bool, default=False)
        """
        self.mega_class = model_class
        self.model = model_class.model
        #print(X_test.shape)
        self.X_test = X_test if X_test.any() else model_class.X_test
        self.Y_test = Y_test if Y_test.any() else model_class.Y_test
        self.cv = cv

    def on_epoch_end(self, epoch, logs={}):
        """
        Callback function called at the end of each epoch during model training.

        Parameters:
        - epoch: Current epoch number (int)
        - logs: Dictionary of logs and metrics (dict)
        """
        self.Y_pred = self.model.predict(self.X_test, verbose = 0)
        #print(self.X_test.shape)
        #print(self.Y_test[:5])
        print("Y_pred shape is {}".format(self.Y_pred.shape))
        mse = mean_squared_error(self.Y_test, self.Y_pred)
        if self.cv:
            self.mega_class.temp_val_scores.append(mse)
        else:
            self.mega_class.test_loss_list.append(mse)
            self.mega_class.pred_per_epoch.append(self.Y_pred)
        
        

"""class CustomCallback(keras.callbacks.Callback):
    def __init__(self, model_class):
        self.mega_class = model_class
        self.model = model_class.model
        self.X_test = model_class.X_test
        self.Y_test = model_class.Y_test

    def on_epoch_end(self, epoch, logs={}):
        self.Y_pred = self.model.predict(self.X_test)
        mse = mean_squared_error(self.Y_test, self.Y_pred)
        self.mega_class.test_loss_list.append(mse)
        self.mega_class.pred_per_epoch.append(self.Y_pred)"""

In [6]:
sheet_name = ['20220524','20220823','20221122','20230221','20230523']
symbols = pd.read_excel("../output/target_stock_universe.xlsx", sheet_name = sheet_name[0]) # change here to get the other sheet
symbols['Ticker_Symbol'] = symbols['Ticker'].str.split().str[0]+".L"
symbols = symbols[symbols['Ticker_Symbol']!='HSV.L']
symbols = symbols[symbols['Ticker_Symbol']!='HBR.L']


symbols_short = symbols['Ticker_Symbol'].iloc[10:] # change here to get the full list of stock
data = price_volume_data(symbols_short.to_list(), '2017-05-25', '2022-05-31')
data.get_technical_indicators()
technical_indicators = ['Close', 'Volume', 'SMA_5', 'SMA_14', 'ATR_5', 'ATR_14', 'ADX_5', 'ADX_14', 'RSI_5',
       'RSI_14', 'MACD', 'MACD_signal', 'BBANDS_upp', 'BBANDS_mid', 'BBANDS_low']

In [7]:
def get_technicals(technical_indicators, symbols_short, data):
    results = {}
    Y = {}
    X = {}
    for x in range(len(symbols_short)):
        results[x] = data.pv[f'{symbols_short.iloc[x]}']
        Y[x] = np.array(results[x]['Close']).reshape(-1,1)
        X[x] = np.array(results[x][technical_indicators])
    return results, Y, X

In [8]:
def run_LSTM(X,Y):
    results = {}
    for x in range(len(symbols_short)):
        model1 = LSTM_predictor(X[x], Y[x])
        model1.prepare_data(train_test_split = 0.8, timestep = 30, look_forward = 3, inplace = True)
        model1.init_model(optimizer='adam', loss='mean_squared_error')
        model1.cross_validation()
        model1.fit_model_train(epochs=30,batch_size=32)
        #model1.fit_model_train(epochs=30,batch_size=32,callbacks=[CustomCallback(model1)])
        model1.predict()
        model1.key_metrics()
        model1.plot_prediction()
        results[x] = model1.key_metrics()
    return results
    

In [9]:
results, Y, X = get_technicals(technical_indicators, symbols_short, data)

In [10]:
Y

{0: array([[322.63446045],
        [328.12503052],
        [325.7718811 ],
        ...,
        [413.82745361],
        [425.50585938],
        [433.47277832]]),
 1: array([[143.1204071 ],
        [142.98141479],
        [141.24363708],
        ...,
        [ 76.01734924],
        [ 74.19231415],
        [ 74.38442993]]),
 2: array([[274.88912964],
        [278.00698853],
        [281.47119141],
        ...,
        [210.64622498],
        [207.9256897 ],
        [212.97807312]]),
 3: array([[ 528.45831299],
        [ 534.33007812],
        [ 528.03900146],
        ...,
        [1066.5826416 ],
        [1076.13134766],
        [1094.27380371]]),
 4: array([[2538.95361328],
        [2545.53393555],
        [2611.33105469],
        ...,
        [2117.06494141],
        [1974.12084961],
        [1990.93774414]]),
 5: array([[1640.73413086],
        [1621.66577148],
        [1621.66577148],
        ...,
        [1512.0447998 ],
        [1529.53625488],
        [1567.92041016]]),
 6: array(

In [75]:
Y[0].shape

(1268, 1)

In [76]:
X[0].shape

(1268, 15)

results

model1 = LSTM_predictor(X1, Y1)
model1.prepare_data(train_test_split = 0.8, timestep = 30, look_forward = 3, inplace = True)
model1.init_model(optimizer='adam', loss='mean_squared_error')
model1.cross_validation()

In [77]:
metrics = run_LSTM(X,Y)

x_train after prepare_data has shape of (1199, 30, 15)
x_test after prepare_data has shape of (15,)
Initiate cross-validation
Running cross validation round 1
Epoch 1/10


2023-10-10 00:34:26.191634: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [204]
	 [[{{node Placeholder/_11}}]]
2023-10-10 00:34:26.191843: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [204]
	 [[{{node Placeholder/_11}}]]


Y_pred shape is (199, 1)
3/3 - 1s - loss: 0.1897 - 1s/epoch - 413ms/step
Epoch 2/10


2023-10-10 00:34:27.310106: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [199,30,15]
	 [[{{node Placeholder/_10}}]]
2023-10-10 00:34:27.310277: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [199,30,15]
	 [[{{node Placeholder/_10}}]]


Y_pred shape is (199, 1)
3/3 - 0s - loss: 0.1303 - 128ms/epoch - 43ms/step
Epoch 3/10
Y_pred shape is (199, 1)
3/3 - 0s - loss: 0.0496 - 126ms/epoch - 42ms/step
Epoch 4/10


...

Y_pred shape is (199, 1)
10/10 - 0s - loss: 0.0092 - 380ms/epoch - 38ms/step
Epoch 1/30
 4/38 [==>...........................] - ETA: 0s - loss: 0.0366

2023-10-10 00:34:40.209342: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [199,30,15]
	 [[{{node Placeholder/_10}}]]
2023-10-10 00:34:40.209517: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [199,30,15]
	 [[{{node Placeholder/_10}}]]
2023-10-10 00:34:40.281214: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and

38/38 [==============================] - 1s 18ms/step - loss: 0.0207
Epoch 2/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0151
Epoch 3/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0139
Epoch 4/30
38/38 [==============================] - 1s 19ms/step - loss: 0.0122
Epoch 5/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0115
Epoch 6/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0118
Epoch 7/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0103
Epoch 8/30
38/38 [==============================] - 1s 19ms/step - loss: 0.0097
Epoch 9/30
38/38 [==============================] - 1s 19ms/step - loss: 0.0107
Epoch 10/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0091
Epoch 11/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0085
Epoch 12/30
38/38 [==============================] - 1s 18ms/step - loss: 0.0085
Epoch 13/30
38/38 [=============================

2023-10-10 00:35:01.544826: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [15]
	 [[{{node Placeholder/_10}}]]
2023-10-10 00:35:01.545015: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [15]
	 [[{{node Placeholder/_10}}]]


TypeError: in user code:

    File "/Users/stevenlee/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/Users/stevenlee/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/stevenlee/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/stevenlee/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/Users/stevenlee/anaconda3/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/stevenlee/anaconda3/lib/python3.10/site-packages/keras/src/layers/r

In [ ]:
metrics

In [ ]:
#################

In [ ]:
# Access train loss values
train_loss_values = model.cv_train_scores

# Access validation loss values
val_loss_values = model.cv_val_scores

min_val_loss = np.argmin(val_loss_values)+1


plt.figure(figsize=(10, 6))
# Create a plot of train and test loss
plt.plot(np.arange(1,51), train_loss_values, label='Train Loss')
plt.plot(np.arange(1,51), val_loss_values, label='Validation Loss')cv_train_scores
plt.axvline(x = min_val_loss, ls = '--', c = 'r', label = 'Min Validation Loss')

# Set plot labels and title
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')

plt.ylim(0,0.5)
plt.xticks(np.arange(1,51,2))

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
val_loss_values.min()

In [ ]:
model.fit_model(epochs=min_val_loss,batch_size=100)
model.predict()
model.plot_prediction()
model.key_metrics()